In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 64)
pd.set_option('display.max_rows', 100)

In [3]:
homePath = 'game'
trainPath = os.path.join(homePath, 'train_n.csv')
trainData = pd.read_csv(trainPath)
testPath = os.path.join(homePath, 'test_n.csv')
testData = pd.read_csv(testPath)

In [4]:
trainLabel = trainData['acc_now_delinq']
testLabel = testData['acc_now_delinq']
del trainData['acc_now_delinq']
del trainData['member_id']
del trainData['index']
del testData['acc_now_delinq']
del testData['member_id']
del testData['index']

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=50, n_jobs=-1)
scores = cross_val_score(rnd_clf, trainData, trainLabel,
                         scoring="recall", cv=10)
scores.mean()

0.78409689601178967

In [6]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
etr_clf = ExtraTreesClassifier(n_estimators=500, max_leaf_nodes=50, n_jobs=-1)
# etr_score = cross_val_score(etr_clf, trainData, trainLabel, scoring='recall', cv=10)
# etr_score.mean()

In [7]:
etr_clf.fit(trainData, trainLabel)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=50,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [8]:
from sklearn.metrics import precision_score, recall_score
def calc_f2(label, predict):
    p = precision_score(label, predict)
    r = recall_score(label, predict)
    f2_score = 5*p*r / (4*p + r)
    return f2_score

In [12]:
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
bst = xgb.XGBClassifier(max_depth=6, min_samples_leaf=2, n_estimators=100, num_round=5, objective='binary:logistic', n_jobs=-1)

from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=50, n_jobs=-1)

from sklearn.ensemble import ExtraTreesClassifier
etr_clf = ExtraTreesClassifier(n_estimators=500, max_leaf_nodes=50, n_jobs=-1)

from sklearn.ensemble import GradientBoostingClassifier
gbdt = GradientBoostingClassifier(learning_rate=0.1,min_samples_leaf=2,max_depth=6,n_estimators=100)

vot = VotingClassifier(estimators=[('bst', bst), ('rnd_clf', rnd_clf), ('etr_clf', etr_clf), ('gbdt', gbdt)], voting='hard')
vot.fit(trainData,trainLabel)


VotingClassifier(estimators=[('bst', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, min_samples_leaf=2, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None, num_round=5,
 ...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [14]:
pred = vot.predict(trainData)
vot_f2 = calc_f2(trainLabel, pred)
vot_f2

0.87626014261126139